In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import train_test_split
from string import punctuation
from collections import Counter
import numpy as np
stop_words = list(stop_words.ENGLISH_STOP_WORDS) + list(punctuation)

In [2]:
hparams = {
    'n_features' : 10000
}

In [17]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

train_data, val_data = train_test_split(list(zip(train.data, train.target)), test_size=0.1, random_state=0)
train_texts, train_targets = zip(*train_data)
val_texts, val_targets = zip(*val_data)

tfidf = TfidfVectorizer(max_features=hparams['n_features'], stop_words=stop_words)
train_features = tfidf.fit_transform(train_texts).toarray().astype(np.float32)
val_features = tfidf.transform(val_texts).toarray().astype(np.float32)
test_features = tfidf.transform(test.data).toarray().astype(np.float32)

class_counts = Counter(train_targets)
class_weights = np.array([max(class_counts.values())/class_counts[i] for i in range(len(class_counts))])

label_dict = dict([(k, v) for k, v in enumerate(train.target_names)])

### Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
clf = LogisticRegression(class_weight='balanced')

In [20]:
clf.fit(train_features, train_targets)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [21]:
clf.score(train_features, train_targets)

0.9643133355430339

In [22]:
clf.score(val_features, val_targets)

0.8824569155987627

In [23]:
clf.score(test_features, test.target)

0.8068242166755177